<a href="https://colab.research.google.com/github/zahraDehghanian97/Apriori/blob/master/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load data

In [31]:
!wget https://github.com/zahraDehghanian97/Apriori/blob/master/records.txt

--2022-12-06 09:48:58--  https://github.com/zahraDehghanian97/Apriori/blob/master/records.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘records.txt’

records.txt             [ <=>                ] 140.25K  --.-KB/s    in 0.01s   

2022-12-06 09:48:58 (14.0 MB/s) - ‘records.txt’ saved [143614]



In [32]:
import pandas as pd
items = []
extended = []
with open("records.txt") as file:
    for line in file:
        s = line.split(" ")
        items.append(s)
        extended.extend(s)
print(len(items))
unique_items = list(set(extended))
print(len(unique_items))

1607
2939


<h2>Convert Items to Number</h2>

In [33]:
dictionaries =dict(zip(tuple(unique_items), tuple(range(len(unique_items)))))
numbered_items = []
for i in range(len(items)):
  temp = []
  for j in items[i] :
    temp.append(dictionaries[j])
  numbered_items.append(list(set(temp)))

print(numbered_items[:10])


[[1582], [1582], [1582], [1582], [1582], [1582], [1582], [1582], [585, 1844], [848, 914, 2419, 2771, 1429, 2810]]


<h2>Change Categorical data into numbered</h2>

In [34]:
dict_data = {'items': numbered_items}
df = pd.DataFrame.from_dict(dict_data)
items = numbered_items
df

,items
0,[1582]
1,[1582]
2,[1582]
3,[1582]
4,[1582]
...,...
1602,[1582]
1603,"[0, 1986, 2860, 1549, 1107, 2135]"
1604,"[0, 2096]"
1605,[1242]


<h2>First Frequent Itemset (L1)</h2>

In [35]:
def count_support(database_dataframe, candidate_list):
    count_candidate = []
    #Set the Initial value of Candidate
    for i in range(len(candidate_list)):
        count_candidate.append((candidate_list[i], 0))
    df_candidate = pd.DataFrame(count_candidate, columns=['itemset', 'sup'])
    for i in range(len(database_dataframe)):
        for j in range(len(count_candidate)):
            if (set(df_candidate['itemset'][j])).issubset(set(database_dataframe['items'][i])): 
                df_candidate.loc[j, 'sup'] += 1
    return df_candidate

In [36]:
candidate1 = []
for i in range(len(unique_items)): candidate1.append([i])
count_candidate1_df = count_support(df,candidate1 )

In [37]:
def filter_sup(candidate):
    minimum_sup = 25
    filtering = candidate['sup'] > minimum_sup
    freq = candidate[filtering]
    return freq

In [38]:
freq_itemset1 = filter_sup(count_candidate1_df)
freq_itemset1_reset = freq_itemset1.reset_index(drop=True)
freq_itemset1=freq_itemset1_reset
freq_itemset1

,itemset,sup
0,[0],1055
1,[93],33
2,[117],34
3,[177],30
4,[248],27
5,[298],41
6,[410],30
7,[581],66
8,[703],27
9,[733],37


# Second Frequent Itemset (L2)


In [39]:
import numpy
def self_join(prev_freq_itemset):
    self_join_candidate = []
    for i in range(len(prev_freq_itemset['itemset'])):
        for j in range((i+1), len(prev_freq_itemset['itemset'])):
            itemset_i = prev_freq_itemset['itemset'][i]
            itemset_j = prev_freq_itemset['itemset'][j]
            if(type(itemset_i) == numpy.int64 and type(itemset_j) == numpy.int64):
                itemset_i = {itemset_i}
                itemset_j = {itemset_j}
            union_candidate = itemset_i.union(itemset_j)

            if union_candidate not in self_join_candidate:
                self_join_candidate.append(union_candidate)
    return self_join_candidate

In [40]:
import numpy
def join(itemset1,itemset2):
    join_candidate = []
    for i in range(len(itemset1['itemset'])):
        for j in range(len(itemset2['itemset'])):
            itemset_i = set(itemset1['itemset'][i])
            itemset_j = set(itemset2['itemset'][j])
            union_candidate = itemset_i.union(itemset_j)
            if ((len(list(union_candidate))== (len(list(itemset_i))+len(list(itemset_j)))) and (union_candidate not in join_candidate)):
                join_candidate.append(list(union_candidate))
    return join_candidate

In [41]:
candidate2_list = join(freq_itemset1,freq_itemset1)
candidate2_list

[[0, 93],
 [0, 117],
 [0, 177],
 [0, 248],
 [0, 298],
 [0, 410],
 [0, 581],
 [0, 703],
 [0, 733],
 [0, 830],
 [0, 996],
 [0, 1029],
 [0, 1064],
 [0, 1086],
 [0, 1166],
 [0, 1175],
 [0, 1184],
 [0, 1309],
 [0, 1499],
 [0, 1549],
 [0, 1582],
 [0, 1639],
 [0, 1655],
 [0, 1670],
 [0, 1734],
 [0, 1793],
 [0, 1819],
 [0, 1933],
 [0, 1984],
 [0, 2245],
 [0, 2346],
 [0, 2486],
 [0, 2515],
 [0, 2523],
 [0, 2531],
 [0, 2593],
 [0, 2594],
 [0, 2635],
 [0, 2722],
 [0, 2779],
 [0, 2794],
 [0, 2814],
 [0, 2924],
 [0, 93],
 [117, 93],
 [177, 93],
 [248, 93],
 [298, 93],
 [410, 93],
 [581, 93],
 [93, 703],
 [733, 93],
 [93, 830],
 [996, 93],
 [1029, 93],
 [1064, 93],
 [93, 1086],
 [93, 1166],
 [93, 1175],
 [1184, 93],
 [1309, 93],
 [1499, 93],
 [1549, 93],
 [93, 1582],
 [93, 1639],
 [93, 1655],
 [93, 1670],
 [93, 1734],
 [1793, 93],
 [1819, 93],
 [93, 1933],
 [1984, 93],
 [2245, 93],
 [2346, 93],
 [93, 2486],
 [2515, 93],
 [2523, 93],
 [2531, 93],
 [2593, 93],
 [2594, 93],
 [2635, 93],
 [2722, 93],
 [

In [ ]:
count_candidate2_df = count_support(df, candidate2_list)
freq_itemset2 = filter_sup(count_candidate2_df)
freq_itemset2 = freq_itemset2.reset_index(drop=True)
freq_itemset2

<h2>Third Frequent Itemset (L3)</h2>

In [ ]:
join_result = join(freq_itemset2,freq_itemset1)
print('join result : ')
print(join_result)

## Pruning

In [ ]:
def get_subset(candidate):
    temp = []
    final = []
    for i in range(len(candidate)):
        for j in range(len(candidate)):
            if i != j:
                temp.append(candidate[j])
        temp_set = set(temp)
        final.append(temp_set)
        temp.clear()
    print('Subset from {} : {}'.format(candidate, final))
    return final

def pruning(candidate_set, prev_freq_itemset):
    print('Candidate set', candidate_set)
    temp = []
    
    for idx, value in enumerate(candidate_set):
        list_candidate = list(value)
        temp_candidate = (get_subset(list_candidate))
        
        for temp_item in temp_candidate:
            print('Temp item', temp_item)
            check = temp_item == prev_freq_itemset['itemset']
            print('\nCheck candidate from Previous Frequent Itemset\n', check)
            
            if any(check) == False:
                print(any(check))
                print('Val', value)
            else:
                print('\nAll of {} subset contained in \n{}'.format(candidate_set, prev_freq_itemset))
                if value not in temp:
                    temp.append(value)
                
    return temp

In [ ]:
candidate3_list = pruning(join_result, freq_itemset2)
print("final candidate 3 list : ")
candidate3_list

In [ ]:
count_candidate3_df = count_support(df, candidate3_list)
freq_itemset3 = filter_sup(count_candidate3_df)
freq_itemset3

## All Frequent Itemset

In [ ]:
frequent_itemset = pd.concat([freq_itemset1, freq_itemset2, freq_itemset3], axis=0)
frequent_itemset_final = frequent_itemset.reset_index(drop=True)
frequent_itemset_final

# Find Association Rules

In [ ]:
for idx, value in enumerate(freq_itemset3):
    list_candidate = list(value['itemset'])
    temp_candidate = (get_subset(list_candidate))
    df_candidate = freq_itemset2[freq_itemset2['itemset']==temp_candidate]
    min_support_itemset =df_candidate[df_candidate['sup']==min(df_candidate['sup'])]
    print(str(min_support_itemset['itemset'])+" ->"+(str(list_candidate.remove(list(min_support_itemset['itemset']))))+" , confidence score : "+str(value['sup']/min_support_itemset['sup']))
for idx, value in enumerate(freq_itemset2):
    list_candidate = list(value['itemset'])
    temp_candidate = (get_subset(list_candidate))
    df_candidate = freq_itemset1[freq_itemset2['itemset']==temp_candidate]
    min_support_itemset =df_candidate[df_candidate['sup']==min(df_candidate['sup'])]
    print(str(min_support_itemset['itemset'])+" ->"+(str(list_candidate.remove(list(min_support_itemset['itemset']))))+" , confidence score : "+str(value['sup']/min_support_itemset['sup']))
